## All

In [45]:
plik = '2475'

In [46]:
import pandas as pd
from decimal import Decimal, ROUND_HALF_UP

In [47]:
def format_to_two_decimals(value):
    return Decimal(value).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)

In [48]:
bc = pd.read_excel(f'{plik}.xlsx', sheet_name='BC', header=0)
# column 'data' is a date column
# bc['debet'] and bc['kredyt'] are decimals - but some of them contain spaces or other white characters inside. ignore these characters
bc['data'] = pd.to_datetime(bc['data'], format='%Y-%m-%d')
bc['debet'] = bc['debet'].astype(str)
bc['kredyt'] = bc['kredyt'].astype(str)
bc['debet'] = bc['debet'].str.replace(r'\s', '', regex=True)
bc['debet'] = bc['debet'].str.replace(r',', '.', regex=True)
bc['kredyt'] = bc['kredyt'].str.replace(r'\s', '', regex=True)
bc['kredyt'] = bc['kredyt'].str.replace(r',', '.', regex=True)
bc['debet'] = bc['debet'].apply(lambda x: format_to_two_decimals(x) if x else Decimal('0'))
bc['kredyt'] = bc['kredyt'].apply(lambda x: format_to_two_decimals(x) if x else Decimal('0'))

## Santander

In [49]:
bank = pd.read_excel(f'{plik}.xlsx', sheet_name='bank', header=0)

In [50]:
bank.head()

,Firma,Rachunek,Data księgowania,Data przyjęcia do realizacji,Strona transakcji,Rachunek strony transakcji,Tytuł,Kwota,Waluta,Typ operacji
0,OREX TRAVEL SP Z O O,76 1090 1841 0000 0001 5424 2475,2024-08-21,2024-08-21,PAWEŁ MICHAŁ STEC UL. CICHA 1B/26 67-100 NOWA SÓL,93 1090 1577 0000 0001 4854 1842,1265801 EIN27109000045747000001265801,8644.0,PLN,UZNANIE
1,OREX TRAVEL SP Z O O,76 1090 1841 0000 0001 5424 2475,2024-08-21,2024-08-21,BIURO PODRÓŻY IWONA IWONA SZUKAŁA-KUŚ UL. GORZ...,17 1090 2138 0000 0001 1216 9318,1261064 EIN80109000045747000001261064,50.0,PLN,UZNANIE
2,OREX TRAVEL SP Z O O,76 1090 1841 0000 0001 5424 2475,2024-08-21,2024-08-21,BIURO PODRÓŻY IWONA IWONA SZUKAŁA-KUŚ UL. GORZ...,17 1090 2138 0000 0001 1216 9318,1261063 EIN10109000045747000001261063,50.0,PLN,UZNANIE
3,OREX TRAVEL SP Z O O,76 1090 1841 0000 0001 5424 2475,2024-08-21,2024-08-21,BIURO PODRÓŻY IWONA IWONA SZUKAŁA-KUŚ UL. GORZ...,17 1090 2138 0000 0001 1216 9318,126 1059 EIN21109000045747000001261059,50.0,PLN,UZNANIE
4,OREX TRAVEL SP Z O O,76 1090 1841 0000 0001 5424 2475,2024-08-21,2024-08-21,CENTRUM USŁUG ROZLICZENIOWYCH,06 1090 0004 0000 0011 7459 0049,2408210857873888REF.PROWIZJA DODATK.DO POLECEN...,-15.0,PLN,OPŁATA/PROWIZJA


In [51]:
# add a column to bank 'co' with values: 'uznanie' if 'kwota' is positive, 'obciążenie' if 'kwota' is negative
bank['co'] = bank['Kwota'].apply(lambda x: 'uznanie' if x > 0 else 'obciążenie')

In [52]:
# create a new dataframe from bank. group by 'Data księgowania' and make 2 columns 'Uznania' and 'Obciążenia' summing up the values of 'Kwota' for column 'co' having value of 'uznanie' or 'obciążenie' respectively
sant = bank.groupby('Data księgowania').apply(lambda x: pd.Series({'Uznania': x[x['co'] == 'uznanie']['Kwota'].sum(), 'Obciążenia': x[x['co'] == 'obciążenie']['Kwota'].sum()})).reset_index()

C:\Users\Kamil Tomaszek\AppData\Local\Temp\ipykernel_14920\2197455804.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sant = bank.groupby('Data księgowania').apply(lambda x: pd.Series({'Uznania': x[x['co'] == 'uznanie']['Kwota'].sum(), 'Obciążenia': x[x['co'] == 'obciążenie']['Kwota'].sum()})).reset_index()


In [53]:
sant['Obciążenia'] = sant['Obciążenia'].abs()
sant['Uznania'] = sant['Uznania'].apply(lambda x: format_to_two_decimals(x) if x else Decimal('0'))
sant['Obciążenia'] = sant['Obciążenia'].apply(lambda x: format_to_two_decimals(x) if x else Decimal('0'))

In [54]:
# change type of data in column 'Data księgowania' to datetime
sant['Data księgowania'] = pd.to_datetime(sant['Data księgowania'], format='%Y-%m-%d')

In [55]:
# now outer merge the two dataframes on the date. let the new dataframe be 'merged' and contain: 'data', 'kredyt_bc', 'debet_bc', 'uznania_bank', 'obciazenia_bank'
merged = pd.merge(bc, sant, left_on='data', right_on='Data księgowania', how='outer')


In [56]:
# change the columns a little bit: i want them in these order: 'data' (which should be 'data' and 'Data Księgowania merged), 'debet_bc', 'obciążenia_bank', 'kredyt_bc', 'uznania_bank'
# if data is null in 'data' column, fill it with 'Data księgowania' column
merged['data'] = merged['data'].combine_first(merged['Data księgowania'])
merged = merged[['data', 'debet', 'kredyt', 'Obciążenia',  'Uznania']]
merged.columns = ['data', 'debet_bc', 'kredyt_bc', 'obciazenia_bank', 'uznania_bank']
# fill NaNs win last 4 columns with 0
merged = merged.fillna(0)

## BNP

In [29]:
bnp = pd.read_excel(f'{plik}.xlsx', sheet_name='bank', header=0)

In [30]:
# change colnames to data, Obciążenia, Uznania
bnp.columns = ['data', 'Obciążenia', 'Uznania', 'waluta']

ValueError: Length mismatch: Expected axis has 10 elements, new values have 4 elements

In [ ]:
bnp['Obciążenia'] = bnp['Obciążenia'].apply(lambda x: format_to_two_decimals(x) if x else Decimal('0'))
bnp['Uznania'] = bnp['Uznania'].apply(lambda x: format_to_two_decimals(x) if x else Decimal('0'))

In [ ]:
merged = pd.merge(bc, bnp, left_on='data', right_on='data', how='outer')

In [ ]:
merged = merged[['data', 'debet', 'kredyt', 'Obciążenia',  'Uznania']]
merged.columns = ['data', 'debet_bc', 'kredyt_bc', 'obciazenia_bank', 'uznania_bank']
merged = merged.fillna(0)

## All

In [57]:
# change data format to 'YYYY-MM-DD'
merged['data'] = merged['data'].dt.strftime('%Y-%m-%d')

In [58]:
# add columns 'debet-obciazenia' and 'kredyt-uznania'
merged['debet-obciazenia'] = merged['debet_bc'] - merged['obciazenia_bank']
merged['kredyt-uznania'] = merged['kredyt_bc'] - merged['uznania_bank']

In [59]:
# save to excel

merged.to_excel(f'merged_{plik}.xlsx', index=False)